## Mandatory

In [66]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic 
import pandas as pd

In [45]:
lemmas = [('the','DT'), ('man','NN'), ('swim','VB'), ('with', 'PR'), ('a', 'DT'),
('girl','NN'), ('and', 'CC'), ('a', 'DT'), ('boy', 'NN'), ('whilst', 'PR'),
('the', 'DT'), ('woman', 'NN'), ('walk', 'VB')]

morphy_tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV}
synsets= []
for lemma, pos in lemmas:
    try:
        synsets.append(wn.synsets(lemma, morphy_tag[pos])[0])
    except:
        print ('The next lemma can not be a synset: ', lemma, pos)


The next lemma can not be a synset:  the DT
The next lemma can not be a synset:  with PR
The next lemma can not be a synset:  a DT
The next lemma can not be a synset:  and CC
The next lemma can not be a synset:  a DT
The next lemma can not be a synset:  whilst PR
The next lemma can not be a synset:  the DT


For each pair, when possible, print their most frequent WordNet synset,
their corresponding least common subsumer (LCS) and their similarity
value, using the following functions:
- Path Similarity
- Leacock-Chodorow Similarity
- Wu-Palmer Similarity
- Lin Similarity

Normalize similarity values when necessary.

In [73]:
print('Most frequent synset:')
print(synsets)

most frequent synset:
[Synset('man.n.01'), Synset('swim.v.01'), Synset('girl.n.01'), Synset('male_child.n.01'), Synset('woman.n.01'), Synset('walk.v.01')]


In [70]:
drug = wn.synset('drug.n.1')
milk = wn.synset('milk.n.1')
drug.lowest_common_hypernyms(milk)
ss_to_text = lambda x: str(x).replace('Synset(','')[1:-7]

common_subsummers = pd.DataFrame()
for synset1 in synsets:
    for synset2 in synsets:
        lcs = synset1.lowest_common_hypernyms(synset2)
        if len(lcs) > 0:
            common_subsummers.loc[ss_to_text(synset1),ss_to_text(synset2)] = ss_to_text(lcs[0])
            #print(ss_to_text(synset1),ss_to_text(synset2), ss_to_text(lcs[0]) )
common_subsummers.fillna('-')                                                                  

,man,girl,male_child,woman,swim,walk
man,man,adult,male,adult,-,-
swim,-,-,-,-,swim,travel
girl,adult,girl,person,woman,-,-
male_child,male,person,male_child,person,-,-
woman,adult,woman,person,woman,-,-
walk,-,-,-,-,travel,walk


In [71]:
simmillarity_functions = [('lch',wn.lch_similarity),('wup', wn.wup_similarity),('path', wn.path_similarity),
                  ('lin',wn.lin_similarity)]

simmilarities = {}

brown_ic = wordnet_ic.ic('ic-brown.dat')

for name, distance in simmillarity_functions:
    simmilarities[name] = pd.DataFrame()
    for synset1 in synsets:
        for synset2 in synsets:
            try:
                if name == 'lin':
                    dis = distance(synset1,synset2, brown_ic)
                elif name == 'lch':
                    dis = distance(synset1,synset2)/distance(synset1,synset1)

                else:
                    dis = distance(synset1,synset2)
            except:
                dis = '-'
            simmilarities[name].loc[ss_to_text(synset1),ss_to_text(synset2)] = dis
    simmilarities[name] = simmilarities[name].fillna('-')


Leacock-Chodorow Similarity

In [72]:
simmilarities['lch']

,man,swim,girl,male_child,woman,walk
man,1,-,0.618897,0.697983,0.697983,-
swim,-,1,-,-,-,0.662805
girl,0.618897,-,1,0.507432,0.809449,-
male_child,0.697983,-,0.507432,1,0.557553,-
woman,0.697983,-,0.809449,0.557553,1,-
walk,-,0.662805,-,-,-,1


Path Similarity

In [60]:
simmilarities['path']

,man,swim,girl,male_child,woman,walk
man,1.000000,-,0.250000,0.333333,0.333333,-
swim,0.100000,1,0.090909,0.100000,0.100000,0.333333
girl,0.250000,-,1.000000,0.166667,0.500000,-
male_child,0.333333,-,0.166667,1.000000,0.200000,-
woman,0.333333,-,0.500000,0.200000,1.000000,-
walk,0.100000,0.333333,0.090909,0.100000,0.100000,1


Wu-Palmer Similarity

In [61]:
simmilarities['wup']

,man,swim,girl,male_child,woman,walk
man,1.000000,-,0.631579,0.666667,0.666667,-
swim,0.181818,1,0.166667,0.181818,0.181818,0.333333
girl,0.631579,-,1.000000,0.631579,0.631579,-
male_child,0.666667,-,0.631579,1.000000,0.666667,-
woman,0.666667,-,0.947368,0.666667,1.000000,-
walk,0.181818,0.333333,0.166667,0.181818,0.181818,1


**Lin Similarity**

In [68]:
simmilarities['lin']

,man,swim,girl,male_child,woman,walk
man,1,-,0.713511,0.729472,0.787084,-
swim,-,1,-,-,-,0.491005
girl,0.713511,-,1,0.292728,0.90678,-
male_child,0.729472,-,0.292728,1,0.318423,-
woman,0.787084,-,0.90678,0.318423,1,-
walk,-,0.491005,-,-,-,1


 **What similarity seems better?**
 
 I'd say Lin is the one tha seems better, as 
 ...
 